<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula13a_huggingface_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  13.7M      0  0:00:05  0:00:05 --:--:-- 18.7M
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [2]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [1]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="SiddhiVisaria/sentiment-analysis-imdb")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
pipe("This is a great movie!")

[{'label': 'LABEL_1', 'score': 0.990974485874176}]

In [9]:
from sklearn.metrics import accuracy_score
import numpy as np
import time

In [ ]:
ypred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_ds):
  for i in range(len(inputs)):
    yp = pipe(inputs[i].numpy().decode('UTF-8')[:512])
    test_labels.append(int(targets[i].numpy()==1))
    ypred.append(int(yp[0]['label']=="LABEL_1"))
  print(batch, time.time()-start, accuracy_score(test_labels, ypred))

0 8.96488094329834 0.90625
1 16.14419460296631 0.90625
2 25.555869102478027 0.8958333333333334
3 35.5209059715271 0.8828125
4 43.56200432777405 0.8875
5 53.022995710372925 0.875
6 61.30147385597229 0.875
7 70.58101654052734 0.85546875
8 79.81586217880249 0.8645833333333334
9 88.83460092544556 0.86875
10 97.52174472808838 0.8664772727272727
11 107.10472393035889 0.859375
12 115.56158542633057 0.8653846153846154
13 124.8377935886383 0.8638392857142857
14 135.30181527137756 0.8541666666666666
15 143.1427617073059 0.85546875
16 152.46549534797668 0.8547794117647058
17 161.3289213180542 0.8489583333333334
18 169.96428322792053 0.850328947368421
19 181.64732933044434 0.8515625
20 192.96901869773865 0.8526785714285714
21 201.45006823539734 0.8508522727272727
22 210.92497563362122 0.842391304347826
23 218.92558407783508 0.8450520833333334
24 228.23095440864563 0.845
25 237.60370540618896 0.8461538461538461
26 245.8617570400238 0.8472222222222222
27 255.37117266654968 0.8470982142857143
28 264.